# Chapter 13: Loading and Preprocessing Data with TensorFlow

Pada praktik Machine Learning modern, sering kali kita berhadapan dengan dataset berukuran sangat besar, bahkan melebihi kapasitas memori (RAM). Oleh karena itu, diperlukan mekanisme yang efisien untuk membaca, memproses, dan mengalirkan data ke model tanpa menyebabkan bottleneck pada proses training.

Bab ini membahas **TensorFlow Data API (`tf.data`)**, sebuah framework powerful yang memungkinkan kita membangun *input pipeline* yang efisien, scalable, dan teroptimasi. Dengan pipeline yang baik, GPU atau TPU dapat bekerja secara maksimal tanpa harus menunggu data disiapkan oleh CPU.

## Daftar Isi:
1. **The Data API**: Konsep dasar dan objek `Dataset`.
2. **Transformasi Data**: Proses chaining, shuffling, batching, dan prefetching.
3. **Membaca CSV Skala Besar**: Strategi membaca banyak file secara paralel.
4. **Format TFRecord**: Format biner efisien untuk data skala besar.
5. **Keras Preprocessing Layers**: Normalisasi dan encoding langsung di dalam model.


## 1. Dasar-Dasar Data API

TensorFlow Data API menyediakan abstraksi tingkat tinggi berupa objek `Dataset`, yang memungkinkan kita membaca data dari berbagai sumber seperti array di memori, file teks, CSV, hingga TFRecord.

Pendekatan ini membuat proses input data:
- Lebih konsisten
- Mudah dioptimalkan
- Cocok untuk pipeline training berskala besar

Dataset bersifat *lazy*, artinya data hanya dibaca ketika dibutuhkan, sehingga sangat efisien dalam penggunaan memori.


In [ ]:
import tensorflow as tf
import numpy as np

# Membuat dataset dari Python list
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

# Menampilkan isi dataset
for item in dataset:
    print(item.numpy())


## 2. Rantai Transformasi (Chaining Transformations)

Salah satu keunggulan utama Data API adalah kemampuannya melakukan transformasi data secara berurutan (*chaining*). Setiap operasi akan menghasilkan objek `Dataset` baru yang dapat diproses lebih lanjut.

Transformasi ini memungkinkan kita:
- Mengulang dataset beberapa kali
- Membagi data ke dalam batch
- Mengontrol alur data secara fleksibel

Pendekatan ini sangat berguna dalam skenario training berulang (epoch).


In [ ]:
# Mengulang dataset 3 kali dan mengelompokkan dalam batch berisi 7
dataset = tf.data.Dataset.range(10)
dataset = dataset.repeat(3).batch(7)

for item in dataset:
    print("Batch:", item.numpy())


### Map, Filter, dan Shuffle

Transformasi lanjutan yang paling sering digunakan meliputi:

- **`map()`**: Mengaplikasikan fungsi ke setiap elemen dataset, misalnya scaling atau normalisasi.
- **`filter()`**: Menyaring data berdasarkan kondisi tertentu.
- **`shuffle()`**: Mengacak urutan data untuk menghindari bias urutan saat training.

Kombinasi ketiga operasi ini sangat penting untuk memastikan model belajar secara stabil dan tidak menghafal urutan data.


In [ ]:
dataset = tf.data.Dataset.range(10)

# Map: kalikan dua, Filter: ambil yang > 10, Shuffle: acak dengan buffer
dataset = dataset.map(lambda x: x * 2) \
                 .filter(lambda x: x > 10) \
                 .shuffle(buffer_size=5, seed=42) \
                 .batch(3)

for item in dataset:
    print("Filtered & Shuffled Batch:", item.numpy())


## 3. Optimasi Pipeline dengan Prefetching

Salah satu penyebab utama training lambat adalah ketidakseimbangan antara CPU dan GPU. Saat GPU sedang idle menunggu data, efisiensi sistem menurun drastis.

**Prefetching** mengatasi masalah ini dengan cara menyiapkan batch berikutnya saat batch saat ini sedang diproses oleh model. Dengan demikian, CPU dan GPU dapat bekerja secara paralel.


In [ ]:
# Gunakan tf.data.AUTOTUNE agar TensorFlow menentukan jumlah prefetch optimal
dataset = tf.data.Dataset.range(10).batch(3).prefetch(tf.data.AUTOTUNE)


## 4. Membaca Banyak File CSV secara Paralel

Pada dataset besar, data sering dibagi ke dalam banyak file CSV. Membaca file secara sekuensial akan memperlambat training.

Dengan menggunakan `interleave`, TensorFlow dapat membaca baris dari beberapa file sekaligus, sehingga throughput data meningkat secara signifikan.


In [ ]:
def parse_line(line):
    # Contoh parsing CSV: 8 fitur float dan 1 label float
    defaults = [0.] * 8 + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defaults)
    return tf.stack(fields[:-1]), fields[-1]

# file_paths = ["file1.csv", "file2.csv", ...]
# dataset = tf.data.Dataset.list_files(file_paths)
# dataset = dataset.interleave(
#     lambda path: tf.data.TextLineDataset(path).skip(1),
#     cycle_length=5)
# dataset = dataset.map(parse_line, num_parallel_calls=tf.data.AUTOTUNE)


## 5. Format Data TFRecord

TFRecord merupakan format penyimpanan biner bawaan TensorFlow yang dirancang untuk efisiensi tinggi. Format ini sangat cocok digunakan ketika dataset berukuran sangat besar atau digunakan dalam sistem terdistribusi.

Keunggulan TFRecord:
- Lebih cepat dibaca dibanding CSV
- Ukuran file lebih kecil
- Terintegrasi langsung dengan ekosistem TensorFlow


In [ ]:
# Menulis TFRecord
with tf.io.TFRecordWriter("data_latihan.tfrecord") as f:
    f.write(b"Data biner pertama")
    f.write(b"Data biner kedua")

# Membaca TFRecord
dataset = tf.data.TFRecordDataset(["data_latihan.tfrecord"])
for item in dataset:
    print(item.numpy())


## 6. Keras Preprocessing Layers

TensorFlow menyediakan preprocessing layer yang dapat dimasukkan langsung ke dalam model. Pendekatan ini membuat preprocessing:
- Konsisten antara training dan inference
- Mudah dipindahkan ke production
- Lebih aman dari data leakage

Layer preprocessing dipelajari menggunakan metode `.adapt()`.


In [ ]:
from tensorflow.keras import layers

# Layer Normalisasi
norm_layer = layers.Normalization()
sample_data = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype="float32")
norm_layer.adapt(sample_data)

# Layer StringLookup (untuk kategori)
lookup = layers.StringLookup()
lookup.adapt(["Kucing", "Anjing", "Burung"])
print("Encoding 'Anjing':", lookup(tf.constant(["Anjing"])).numpy())

# Mengintegrasikan ke dalam model
model = tf.keras.models.Sequential([
    norm_layer,
    layers.Dense(10, activation="relu"),
    layers.Dense(1)
])


## Rangkuman Praktis

Bab ini menekankan pentingnya input pipeline yang efisien dalam Machine Learning skala besar. Beberapa poin penting yang perlu diingat:

1. Gunakan **prefetching** untuk memaksimalkan pemanfaatan hardware.
2. Paralelkan proses `map()` untuk mempercepat preprocessing.
3. Gunakan **TFRecord** untuk performa I/O terbaik.
4. Pastikan preprocessing layer dipelajari menggunakan `.adapt()` sebelum training.

Dengan pipeline data yang baik, proses training menjadi lebih cepat, stabil, dan siap untuk deployment skala produksi.
